# Introduction aux Transformers et LLMs

Les LLMs (Large Language Models) sont des modèles de Deep Learning entraînés sur des quantités massives de texte. Ils sont capables de générer du texte cohérent, de répondre à des questions, de résumer ou de traduire, entre autres. Le secret de leur efficacité réside dans l'utilisation de la structure Transformer, combinée à des embeddings et à des techniques de génération avancées.

Ce notebook illustre les étapes fondamentales pour comprendre et utiliser un LLM en pratique :

1. Tokenisation
2. Embeddings
3. Transformers
4. Utilisation d'un LLM pré-entraîné (différentes tâches)
5. Paramètres de génération (température, longueur de contexte, etc.)

In [ ]:
!pip install transformers tensorflow -q

In [ ]:
import tensorflow as tf
from transformers import (
    AutoTokenizer,
    TFAutoModelForCausalLM,
    pipeline
)

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.1


## 1. Qu'est-ce qu'un LLM ?


Un Large Language Model est un réseau de neurones géant, généralement de type Transformer, pré-entraîné sur des corpus de taille colossale (des milliards de tokens). En termes simples, il apprend à prédire le token (mot, sous-mot, symbole) suivant en se basant sur le contexte précédent.

Grâce à ce principe, il est possible de l’utiliser pour :

1. Génération de texte : production de texte cohérent à partir d’un prompt.
2. Traduction : transformer un texte d’une langue à une autre.
3. Résumé : condenser de longs documents en quelques lignes.
4. Q&A (Question Answering) : répondre à des questions précises.
5. Classification : affecter une étiquette à un texte (sentiment, sujet, etc.).
6. Analyse de similarité sémantique : comparer des phrases ou des mots pour mesurer leur proximité.


## 2. La tokenisation


La tokenisation consiste à convertir un texte brut en une liste de tokens (sous-unités textuelles). Les modèles modernes utilisent souvent des algorithmes comme Byte Pair Encoding (BPE) ou WordPiece, qui découpent les mots rares en morceaux pour mieux gérer le vocabulaire.

**Pourquoi est-ce crucial ?**

- Adapter la forme textuelle brute en format compréhensible pour le modèle.
- Gérer les mots inconnus ou très rares en les décomposant.
- Réduire la taille de vocabulaire tout en conservant une bonne couverture linguistique.

In [ ]:
from transformers import AutoTokenizer

# Chargement d'un tokenizer GPT-2
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Définir un token de padding (ici on utilise le token de fin de séquence comme padding)
tokenizer.pad_token = tokenizer.eos_token

# Exemple de phrases en anglais
sentences = [
    "Hello, how are you?",
    "I love pizza!",
    "The cat sat on the mat.",
    "Transformers are awesome."
]

# Encodage (tokenisation)
encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='tf')

In [ ]:
print("Tokens encodés (IDs) :")
print(encoded_inputs['input_ids'])
print("\nShape : ", encoded_inputs['input_ids'].shape)

# Décodage inverse d'un exemple
decoded_sentence = tokenizer.decode(encoded_inputs['input_ids'][0])
print("\nExemple de décodage inverse :")
print(decoded_sentence)

Tokens encodés (IDs) :
tf.Tensor(
[[15496    11   703   389   345    30 50256]
 [   40  1842 14256     0 50256 50256 50256]
 [  464  3797  3332   319   262  2603    13]
 [41762   364   389  7427    13 50256 50256]], shape=(4, 7), dtype=int32)

Shape :  (4, 7)

Exemple de décodage inverse :
Hello, how are you?<|endoftext|>


Ici, on importe le tokenizer GPT-2 et on lui passe une liste de phrases en anglais. Le tokenizer convertit chacune de ces phrases en une séquence d’IDs (des entiers qui représentent les tokens).

- return_tensors='tf' crée des tenseurs directement utilisables par TensorFlow.
- padding=True et truncation=True gèrent la longueur des séquences : elles sont remplies ou coupées pour correspondre à la taille maximale requise.
La dernière ligne effectue un décodage inverse pour montrer comment les IDs sont reconvertis en texte.

## 3. Les embeddings


Les embeddings sont des vecteurs denses qui représentent la signification ou la sémantique des tokens. Deux tokens ayant un sens proche auront des vecteurs similaires dans cet espace vectoriel.

**Pourquoi est-ce important ?**

Permet la généralisation : un ID numérique seul est moins riche qu’un vecteur qui encode des similarités sémantiques.
Agit comme point de départ pour les couches suivantes du modèle (notamment le Transformer).

In [ ]:
# Chargement du modèle GPT-2 (TensorFlow)
gpt2_model = TFAutoModelForCausalLM.from_pretrained("gpt2", from_pt=True)

# On prépare les données (déjà tokenisées plus haut)
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

# On effectue une passe avant en activant la récupération des états cachés
outputs = gpt2_model(
    input_ids,
    attention_mask=attention_mask,
    output_hidden_states=True
)

# hidden_states est un tuple contenant l'embedding initial à l'indice [0]
all_hidden_states = outputs.hidden_states
embeddings_layer = all_hidden_states[0]

print("Shape du tenseur d'embeddings (batch, seq_len, dim) :")
print(embeddings_layer.shape)

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Shape du tenseur d'embeddings (batch, seq_len, dim) :
(4, 7, 768)


Cette cellule charge le modèle GPT-2. On lui fournit les input_ids et le attention_mask (qui indique quels tokens sont réels vs. pad). On demande au modèle de retourner les hidden_states. La première couche (index 0) contient les embeddings initiaux des tokens. L’affichage montre les dimensions : (taille_batch, longueur_sequence, dimension_embeddings).


In [ ]:
# Exemple de comparaison d'embeddings de deux mots dans des phrases similaires
sent1 = tokenizer("I love pizza", return_tensors="tf")
sent2 = tokenizer("I love pasta", return_tensors="tf")

out1 = gpt2_model(**sent1, output_hidden_states=True).hidden_states[0]
out2 = gpt2_model(**sent2, output_hidden_states=True).hidden_states[0]

embed_pizza = out1[0, -1, :]  # embedding du dernier token "pizza"
embed_pasta = out2[0, -1, :]  # embedding du dernier token "pasta"

# Calcul de la similarité cosinus
cosine_sim = tf.keras.losses.cosine_similarity(embed_pizza, embed_pasta, axis=0)
print("Similitude cosinus (pizza / pasta) :", 1 - cosine_sim.numpy())

Similitude cosinus (pizza / pasta) : 1.8553624749183655


Ici, on compare la similarité cosinus entre les embeddings de “pizza” et “pasta”. Dans l’espace des embeddings, deux termes liés (ici, deux aliments) devraient avoir une similarité plus élevée qu’avec des mots sans rapport.


## 4. Les Transformers


Un Transformer se compose principalement de :

- Mécanisme d’Attention (Self-Attention ou Multi-Head Self-Attention)
- Couches Feed-Forward
- Skip-connections (Add & Norm)
- Normalisation de couche

### 4.1 Le mécanisme d’attention

- Chaque token génère trois vecteurs : Query (Q), Key (K), Value (V).
- On calcule un score de similarité entre Q et tous les K du contexte (autres tokens).
- On combine les valeurs V pondérées par ces scores pour obtenir un vecteur d’attention.
- En Multi-Head, on répète ce processus en parallèle pour capturer des relations variées.

### 4.2 Les couches Feed-Forward

- Chaque token passe ensuite par un réseau fully-connected (MLP) qui affine l’information extraite.
- Les skip-connections (résidus) aident à conserver le signal et à éviter le problème du gradient qui disparaît.
- Les couches de normalisation maintiennent la stabilité de l'entraînement.

### 4.3 Avantages par rapport aux RNN

- Parallélisation : toutes les positions d’une séquence sont traitées en même temps, accélérant l’apprentissage.
- Long contexte : l’attention permet d’exploiter des relations distantes entre tokens (contrairement à un LSTM qui a du mal avec les dépendances trop longues).
- Performance : l'architecture est efficace pour s’entraîner sur de grands volumes de données.

### 4.4 Exemples d'applications avec pipelines

Les pipelines de la bibliothèque transformers facilitent l’utilisation de différents modèles pour diverses tâches : traduction, résumé, génération de texte, Q&A, etc.

In [ ]:
# Exemple 1: Génération de texte (GPT-2)
text_generator = pipeline(
    "text-generation",
    model="gpt2",
    tokenizer="gpt2",
    framework="tf"
)

In [ ]:
prompt = "Artificial intelligence will one day"
generated_texts = text_generator(prompt, max_length=30, num_return_sequences=1)
print("Génération de texte (GPT-2) :")
print(generated_texts[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Génération de texte (GPT-2) :
Artificial intelligence will one day become a significant force in the health and social lives of all living things. It's also one of humanity's major threats


Cette cellule utilise le pipeline text-generation avec le modèle GPT-2. On fournit un prompt en anglais, puis on laisse le modèle générer la suite. max_length=30 limite la génération à 30 tokens supplémentaires.

In [ ]:
# Exemple 2: Résumé (T5)
summarizer = pipeline(
    "summarization",
    model="t5-small",
    tokenizer="t5-small",
    framework="tf"
)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use 0


In [ ]:
long_text = (
    "Natural language processing (NLP) is a subfield of linguistics, "
    "computer science, and artificial intelligence concerned with the "
    "interactions between computers and human language, in particular "
    "how to program computers to process and analyze large amounts of "
    "natural language data."
)

summary = summarizer(long_text, max_length=40, min_length=10, do_sample=False)
print("Résumé du texte :")
print(summary[0]['summary_text'])

Résumé du texte :
natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence . it is concerned with the interactions between computers and human language .


Cette cellule montre l’utilisation du pipeline summarization avec le modèle T5 (en version t5-small). Le texte initial (assez long) est résumé en quelques phrases.

- max_length=40 fixe la longueur maximale du résumé.
- min_length=10 assure qu’il y ait au moins 10 tokens dans la sortie

In [ ]:
# Exemple 3: Traduction (T5)
translator = pipeline(
    "translation_en_to_fr",
    model="t5-small",
    tokenizer="t5-small",
    framework="tf"
)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Device set to use 0


In [ ]:
english_text = "Transformers have revolutionized natural language processing."
translation = translator(english_text, max_length=40)
print("Traduction en français :")
print(translation[0]['translation_text'])

Traduction en français :
Les transformateurs ont révolutionné le traitement de la langue naturelle.


Le pipeline translation_en_to_fr utilise également T5, mais spécialisé pour la traduction de l’anglais vers le français. Le texte en anglais est automatiquement traduit par le modèle.

## 5. Comment utiliser un LLM pré-entraîné


En général, pour utiliser un modèle pré-entraîné, on suit ces étapes :

- Charger le tokenizer (ex. GPT-2, BERT, etc.).
- Charger le modèle correspondant (TFAutoModelForCausalLM, TFAutoModelForSequenceClassification, etc.).
- Préparer le texte sous forme de tokens.
- Effectuer l’inférence (génération, classification, etc.).
- Ajuster les hyperparamètres (température, top-k, top-p, max_length, etc.) selon les besoins de créativité ou de cohérence.

Les exemples de pipeline ci-dessus ont déjà démontré ces étapes de façon simplifiée.

## 6. Paramètres de génération : température et longueur de contexte

### 6.1 Température
- Contrôle l’aléatoire dans la génération.
- Température élevée (ex. 1.3) = plus de diversité, plus de “créativité”.
- Température basse (ex. 0.7) = texte plus conservateur, s’en tenant aux prédictions les plus probables.

### 6.2 Longueur de contexte
- Les Transformers ont une limite en nombre de tokens (ex. ~1024 tokens pour GPT-2).
- Plus le contexte est grand, plus le modèle peut référencer des informations anciennes.
- Une trop grande séquence peut cependant augmenter le coût de calcul et la mémoire requise.

In [ ]:
prompt_temp = "The meaning of life is"
input_ids_temp = tokenizer.encode(prompt_temp, return_tensors="tf")

In [ ]:
for temp in [0.7, 1.0, 1.3]:
    output_temp = gpt2_model.generate(
        input_ids_temp,
        max_length=30,
        temperature=temp,
        do_sample=True,
        num_return_sequences=1
    )
    text_temp = tokenizer.decode(output_temp[0], skip_special_tokens=True)
    print(f"--- Température = {temp} ---")
    print(text_temp)
    print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


--- Température = 0.7 ---
The meaning of life is simple: a person can be anything. The meaning of life is that people have something to say about it. That is why



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


--- Température = 1.0 ---
The meaning of life is not in God (as in nature) but in an infinite mystery that cannot be known, and that is precisely why it calls

--- Température = 1.3 ---
The meaning of life is one whose action constitutes a subject to all laws – but in the past, the Romans could define itself in terms of rules which



Dans ce code, on génère du texte à partir du même prompt mais avec différentes températures (0.7, 1.0, et 1.3). On peut observer la variation de style et de créativité dans le résultat.

- max_length=30 limite la génération.
- do_sample=True indique que le modèle utilise un échantillonnage probabiliste au lieu de toujours prendre le token le plus probable.

## Conclusion

- Les LLMs s'appuient sur l'architecture Transformer pour capter des dépendances à longue portée.
- La tokenisation permet de transformer le texte brut en séquences de tokens exploitables.
- Les embeddings encodent les relations sémantiques entre tokens dans un espace vectoriel.
- Les Transformers utilisent l'attention multi-têtes et des blocs feed-forward pour traiter le contexte global de manière efficace.
- Les pipelines facilitent grandement l’exploitation de modèles pré-entraînés (génération, résumé, traduction...).
- Les paramètres de génération (température, longueur maximale, etc.) influencent la cohérence et l’originalité du texte produit.

Cette exploration offre une base pour aller plus loin dans la personnalisation (fine-tuning) ou le développement d’applications de traitement du langage naturel.